# Docking Analysis Example

Este notebook exemplifica a análise do arquivo `docking_results/summary_affinities.csv` gerado pelo `run_docking.sh`.

Conteúdo:
- Leitura e limpeza do CSV
- Ranking dos melhores ligantes por alvo
- Distribuições de afinidades por alvo (violin/box)
- Heatmap de afinidades (ligantes × alvos)

Pré‑requisitos: `pandas`, `matplotlib` e opcionalmente `seaborn`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# seaborn opcional
try:
    import seaborn as sns
    HAS_SNS = True
except Exception:
    HAS_SNS = False

%matplotlib inline
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = (8, 4)

In [ ]:
from pathlib import Path
summary_path = Path('docking_results/summary_affinities.csv')
assert summary_path.exists(), f'Arquivo não encontrado: {summary_path}'
df = pd.read_csv(summary_path)
# converter afinidade para numérico (NA vira NaN)
df['best_affinity_kcal_per_mol_num'] = pd.to_numeric(df['best_affinity_kcal_per_mol'], errors='coerce')
df.head()

In [ ]:
print('Entradas:', len(df))
print('Alvos:', df['target'].nunique())
print('Ligantes:', df['ligand'].nunique())
print('Registros válidos (afinidade != NA):', df['best_affinity_kcal_per_mol_num'].notna().sum())

## Filtros opcionais
Defina uma lista de alvos para restringir as análises (ou deixe vazia).

In [ ]:
targets_filter = []  # ex.: ['CHS', 'FKS']
if targets_filter:
    df = df[df['target'].isin(targets_filter)].copy()
    print('Filtrado para alvos:', targets_filter, '=>', len(df), 'linhas')
else:
    print('Sem filtro de alvos.')

## Top-N ligantes por alvo
Para cada alvo, exibe os N ligantes com melhor afinidade (mais negativa).

In [ ]:
N = 10  # top-N
best_by_target = []
for target, sub in df.dropna(subset=['best_affinity_kcal_per_mol_num']).groupby('target'):
    sub_sorted = sub.sort_values('best_affinity_kcal_per_mol_num', ascending=True).head(N)
    best_by_target.append((target, sub_sorted[['ligand', 'best_affinity_kcal_per_mol_num']]))

for target, tbl in best_by_target:
    print(f'
== Top {N} para alvo: {target} ==')
    display(tbl.reset_index(drop=True))

## Distribuições por alvo (Violin / Box)

In [ ]:
valid = df.dropna(subset=['best_affinity_kcal_per_mol_num'])
if HAS_SNS:
    fig, ax = plt.subplots(figsize=(10, 4))
    sns.violinplot(data=valid, x='target', y='best_affinity_kcal_per_mol_num', ax=ax, inner='point', cut=0)
    ax.set_title('Distribuição de afinidades por alvo (Violin)')
    ax.set_xlabel('Alvo')
    ax.set_ylabel('Afinidade (kcal/mol), mais negativo é melhor')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    fig, ax = plt.subplots(figsize=(10, 4))
    sns.boxplot(data=valid, x='target', y='best_affinity_kcal_per_mol_num', ax=ax, showmeans=True)
    ax.set_title('Distribuição de afinidades por alvo (Boxplot)')
    ax.set_xlabel('Alvo')
    ax.set_ylabel('Afinidade (kcal/mol), mais negativo é melhor')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    # fallback simples sem seaborn: boxplot de listas
    groups = [g['best_affinity_kcal_per_mol_num'].values for _, g in valid.groupby('target')]
    labels = [t for t, _ in valid.groupby('target')]
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.boxplot(groups, showmeans=True)
    ax.set_title('Distribuição de afinidades por alvo (Boxplot)')
    ax.set_xlabel('Alvo')
    ax.set_ylabel('Afinidade (kcal/mol)')
    ax.set_xticklabels(labels, rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Heatmap (ligantes × alvos)
Usa a melhor afinidade por par (ligante × alvo).

In [ ]:
# reduzir para uma linha por par (ligante, alvo) com a melhor afinidade
grp = (valid.groupby(['ligand','target'])
       ['best_affinity_kcal_per_mol_num']
       .min()
       .reset_index())
pivot = grp.pivot(index='ligand', columns='target', values='best_affinity_kcal_per_mol_num')
pivot = pivot.sort_index(axis=0).sort_index(axis=1)
pivot.head()

In [ ]:
if HAS_SNS:
    plt.figure(figsize=(10, max(4, 0.18*len(pivot))))
    sns.heatmap(pivot, cmap='viridis', annot=False)
    plt.title('Heatmap de afinidades (ligantes × alvos) — mais negativo é melhor')
    plt.xlabel('Alvo')
    plt.ylabel('Ligante')
    plt.tight_layout()
    plt.show()
else:
    plt.figure(figsize=(10, max(4, 0.18*len(pivot))))
    plt.imshow(pivot.values, aspect='auto', cmap='viridis')
    plt.colorbar(label='Afinidade (kcal/mol)')
    plt.title('Heatmap de afinidades (ligantes × alvos) — mais negativo é melhor')
    plt.xlabel('Alvo')
    plt.ylabel('Ligante')
    plt.yticks(range(len(pivot.index)), pivot.index)
    plt.xticks(range(len(pivot.columns)), pivot.columns, rotation=45, ha='right')
    plt.tight_layout()
    plt.show()